# Trabajo de simulación 3

Para el último trabajo de simulación, vamos a hacer un análisis de datos a partir de la encuesta de salarios de Sysarmy del primer trimestre de 2025 (si están publicados los del segundo trimestre o posterior, podemos usar esos, pero a agosto de 2025 todavía no estaban).

Los datos oficiales se encuentran [acá](https://docs.google.com/spreadsheets/d/1hlLwv9SLJvrnsTq_UsEAHkHGNiziH7IdT1lJd4fq6kU/edit?gid=1462536742#gid=1462536742). Por si en algún momento el enlace se cae o cambia de ubicación, una versión alojada en mi drive puede encontrarse [acá](https://docs.google.com/spreadsheets/d/1Vq9F6xE03fR0x6pXlRMDN_bww7NWItjMW0U89qV3AvM/edit?usp=sharing).

Las consignas de este trabajo no son tan dirigidas como las de los trabajos anteriores, pues en el análisis de datos, siempre hay libertad y margen para la creatividad y la producción personal. Sin embargo, les compartimos algunas pautas de lo que debe tener, como mínimo, este trabajo.

Pautas generales y **OBLIGATORIAS** para la aprobación de la entrega:

*   Debe replicarse, como mínimo, un análisis similar al aquí presentado, para estos datos (SysArmy 2025 o cualquier otro dataset que sea de su interés). Por "replicar" nos referimos a que el análisis debe incluir: inspección y limpieza de los datos, descripción y visualización, o estudio de alguna variable de interés a partir de alguna hipótesis o conjetura.

*   Debe escribirse en formato "informe", es decir, no se trata de exhibir código y gráficos, sino de explicar qué se observa y por qué es relevante observar eso. El informe es requisito **excluyente**. No se aprueba el trabajo de simulación sin él. Este informe breve debe entregarse en pdf, en esta entrega **NO** se evalúa el *colab*, sino el reporte. No tiene que ser largo, al contrario, tiene que ser de calidad. Como dice el dicho: lo bueno -si breve- dos veces bueno...

*   El trabajo debe contener, como mínimo, **una conjetura que sea sometida a prueba y de la que se exhiba alguna conclusión fundamentada**, como se hizo en el caso de los datos de 2020 para el salario medio bruto por género y para hombres y mujeres con nivel universitario completo. Por ejemplo, frente a la pregunta de si el salario medio de mujeres y hombres es igual, podríamos poner en práctica lo que estudiamos sobre convergencia para, de alguna forma, darnos una idea de cuán probable es observar lo que efectivamente estamos observando. Este es un "coqueteo" con la estadística inferencial, que no estudiamos formalmente en la materia, pero que es válido comenzar a encarar con todo lo que hemos estudiado. Esta conjetura puede hacerse con datos propios, si es que eligen trabajar con otro dataset.

El resto de la producción queda a criterio de los grupos. Esperamos que haya un interés genuino en tratar de extraer información a partir de estos datos. ¡Muchos éxitos!

**PD: El formato "informe" puede ser cambiado por el formato "póster/infografía" si es que así prefieren.**

# Optimización salarial en el Sector IT

Este análisis está desarrollado desde la perspectiva de un/a profesional del sector IT que necesita reorientar su carrera por diversos motivos. Sabemos que esta industria se caracteriza por una alta rotación laboral, así como por recorridos profesionales diversos y cambiantes.

La pregunta central de nuestro trabajo es:
¿Cómo puedo maximizar mi salario en el sector IT?

Para abordarla, buscamos identificar la combinación ganadora de factores (como tecnologías, roles, experiencia, ubicación, inglés, entre otros) que arroje valor a partir del dataset que analizamos. Este insight puede resultar útil tanto para:

- Perfiles junior, que están ingresando al mercado laboral y necesitan una guía sobre cómo orientar su desarrollo.

- Perfiles senior, que quizás se sienten estancados y buscan estrategias para revalorizarse o replantear su carrera.

In [ ]:
# Importación de librerías necesarias para el análisis
import pandas as pd
import numpy as np
import warnings
import re
from unidecode import unidecode
warnings.filterwarnings('ignore')

# Visualización
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score

# Configuración de Plotly
pio.templates.default = "plotly_white"

## Inspección y limpieza de datos

Comenzamos importando las librerías necesarias y cargando el dataset. Luego inspeccionamos los datos para ver cuántos registros contiene, qué variables tenemos disponibles y qué tipo de datos aportan. Esto nos ayuda a decidir cómo procesarlos más adelante.

In [ ]:
# [PASO 1] Carga del dataset
file_path = 'data/Sysarmy_sueldos_2025.01CLEAN.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
    print(f"*** Archivo cargado exitosamente")
    print(f"    Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas\n")
except FileNotFoundError:
    print(f"ERROR: Archivo no encontrado en '{file_path}'")
    df = None
except Exception as e:
    print(f"ERROR: {e}")
    df = None

if df is None:
    raise Exception("No se pudo cargar el dataset")

# Vista preliminar de las columnas y tipos de datos
df.info()
df.head()


Luego procesamos los datos para dejarlos en un formato adecuado para el análisis. Estas transformaciones incluyen:

- Renombrar columnas para que tengan nombres consistentes.

- Convertir variables numéricas que fueron cargadas como texto.

- Calcular un salario mensual en dólares basado en el tipo de cambio reportado o estimado.

In [ ]:
# [PASO 2] Renombrado de columnas
column_mapping = {
    'donde_estas_trabajando': 'provincia',
    'dedicacion': 'dedicacion',
    'tipo_de_contrato': 'tipo_contrato',
    'ultimo_salario_mensual_o_retiro_bruto_en_pesos_argentinos': 'salario_bruto_ars',
    'ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos': 'salario_neto_ars',
    'pagos_en_dolares': 'forma_pago',
    'trabajo_de': 'puesto',
    'anos_de_experiencia': 'experiencia',
    'antiguedad_en_la_empresa_actual': 'antiguedad',
    'modalidad_de_trabajo': 'modalidad',
    'cantidad_de_personas_en_tu_organizacion': 'tamano_empresa',
    'tengo_edad': 'edad',
    'genero': 'genero',
    'seniority': 'seniority',
    '_sal': 'salario_ars',
    'cuantas_personas_tenes_a_cargo': 'personas_a_cargo',
    'si_tu_sueldo_esta_dolarizado_cual_fue_el_ultimo_valor_del_dolar_que_tomaron': 'tc_reportado'
}

df_clean = df.rename(columns=column_mapping)

# [PASO 3] Conversión de tipos numéricos
df_clean['salario_bruto_ars'] = pd.to_numeric(df_clean['salario_bruto_ars'], errors='coerce')
df_clean['salario_ars'] = pd.to_numeric(df_clean['salario_ars'], errors='coerce')
df_clean['tc_reportado'] = pd.to_numeric(df_clean['tc_reportado'], errors='coerce')

# [PASO 4] Cálculo de salario en USD
print("[PASO 4] Calculando salarios en USD con TC individual...\n")

df_clean['salario_usd'] = np.nan

# Usar TC reportado (prioridad 1)
mask_con_tc = (
    df_clean['tc_reportado'].notna() & 
    (df_clean['tc_reportado'] > 0) & 
    df_clean['salario_ars'].notna()
)

df_clean.loc[mask_con_tc, 'salario_usd'] = (
    df_clean.loc[mask_con_tc, 'salario_ars'] / df_clean.loc[mask_con_tc, 'tc_reportado']
)

print(f"    * {mask_con_tc.sum()} registros con TC reportado individual")

# Calcular y aplicar TC mediano para el resto (prioridad 2)
tc_mediano = df_clean.loc[df_clean['tc_reportado'] > 0, 'tc_reportado'].median()
print(f"    * TC mediano de la encuesta: ${tc_mediano:,.2f}")

mask_sin_tc = df_clean['salario_usd'].isna() & df_clean['salario_ars'].notna()
if mask_sin_tc.sum() > 0:
    df_clean.loc[mask_sin_tc, 'salario_usd'] = (
        df_clean.loc[mask_sin_tc, 'salario_ars'] / tc_mediano
    )
    print(f"    * {mask_sin_tc.sum()} registros usando TC mediano\n")

# [PASO 5] Nuevas variables categóricas derivadas
df_clean['es_caba'] = df_clean['provincia'].apply(
    lambda x: 'CABA' if x == 'Ciudad Autonoma de Buenos Aires' else 'Resto del pais'
)

df_clean['tiene_equipo'] = df_clean['personas_a_cargo'].apply(
    lambda x: 'Con equipo' if pd.notna(x) and x > 0 else 'Sin equipo'
)

print(f"Dataset preparado: {df_clean.shape}")
print(f"Registros con salario USD: {df_clean['salario_usd'].notna().sum()}\n")

Una vez completada la fase de limpieza y preparación del dataset, iniciamos el análisis exploratorio para comprender cómo se distribuyen los salarios según diferentes variables. Este enfoque permite evaluar de manera aislada el efecto de cada dimensión (como ubicación, tipo de contrato, dedicación o seniority) sobre el salario mensual en USD.

A partir de este análisis, podremos identificar "la combinación ganadora" de factores que maximiza el salario, así como caracterizar al profesional mejor pago según los datos de la encuesta.

In [ ]:
# ====================================================================
# 1. Dataset para análisis
# ====================================================================
columnas_analisis = ['salario_usd', 'experiencia', 'es_caba', 'dedicacion',
                     'tipo_contrato', 'puesto', 'seniority', 'modalidad',
                     'edad', 'antiguedad', 'personas_a_cargo', 'forma_pago', 
                     'genero', 'provincia', 'tamano_empresa', 'tiene_equipo']

columnas_analisis = [col for col in columnas_analisis if col in df_clean.columns]
df_analisis = df_clean[columnas_analisis].dropna(subset=['salario_usd']).copy()

print(f"[INFO] Variables analizadas: {len(columnas_analisis)}")
print(f"[INFO] Registros válidos: {len(df_analisis)}\n")

# ====================================================================
# Análisis univariado
# ====================================================================
print("Realizamos un análisis univariado de cada variable para identificar su impacto en el salario.\n")

# Función para analizar una variable categórica
def analizar_variable(df, columna, top_n=None):
    """
    Agrupa por la columna especificada y calcula estadísticos del salario.
    Retorna la tabla de estadísticos y el valor ganador según mediana.
    """
    stats = df.groupby(columna)['salario_usd'].agg([
        ('Media', 'mean'), 
        ('Mediana', 'median'), 
        ('Q25', lambda x: x.quantile(0.25)),
        ('Q75', lambda x: x.quantile(0.75)),
        ('Max', 'max'), 
        ('Cantidad', 'count')
    ]).round(2).sort_values('Mediana', ascending=False)

    if top_n:
        stats = stats.head(top_n)
    
    print(f"[ANÁLISIS] {columna.upper()}:\n")
    print(stats)
    
    mejor_valor = stats['Mediana'].dropna().idxmax()
    print(f"\nGANADOR: {mejor_valor}")
    print(f"Mediana: ${stats.loc[mejor_valor, 'Mediana']:,.0f} USD\n")
    
    return stats, mejor_valor

A continuación, se realiza un análisis univariado de cada variable relevante. 
Esto nos permite identificar cómo cada dimensión (ubicación, tipo de contrato, seniority, etc.) impacta en el salario, 
y comenzar a definir la "combinación ganadora".

In [ ]:
# 1. Ubicación
ubicacion_stats, mejor_ubicacion = analizar_variable(df_analisis, 'es_caba')

# 2. Tipo de Contrato
contrato_stats, mejor_contrato = analizar_variable(df_analisis, 'tipo_contrato')

# 3. Dedicación
dedicacion_stats, mejor_dedicacion = analizar_variable(df_analisis, 'dedicacion')

# 4. Modalidad
modalidad_stats, mejor_modalidad = analizar_variable(df_analisis, 'modalidad')

# 5. Seniority
seniority_stats, mejor_seniority = analizar_variable(df_analisis, 'seniority')

# 6. Puesto (Top 10)
puesto_stats, mejor_puesto = analizar_variable(df_analisis, 'puesto', top_n=10)

# 7. Forma de Pago
pago_stats, mejor_pago = analizar_variable(df_analisis, 'forma_pago')

# 8. Tamaño de Empresa
empresa_stats, mejor_empresa = analizar_variable(df_analisis, 'tamano_empresa')

# 9. Personas a Cargo / Equipo
equipo_stats, mejor_equipo = analizar_variable(df_analisis, 'tiene_equipo')

# El perfil del top 10%
Si bien nuestro objetivo es encontrar la combinación ganadora, creemos que un buen punto de partida es analizar el perfil del top 10% con el mejor salario. La idea es entender qué características tienen en común los profesionales que están en este grupo y ver si podemos extraer alguna conclusión útil para nuestro análisis.

In [ ]:
# ====================================================================
# 2. Perfil del Top 10% mejor pagado
# ====================================================================
print("-"*70)
print("PERFIL DEL TOP 10% MEJOR PAGADO")
print("-"*70 + "\n")

percentil_90 = df_analisis['salario_usd'].quantile(0.90)
top_10_percent = df_analisis[df_analisis['salario_usd'] >= percentil_90].copy()

print(f"*** Salario mínimo Top 10%: ${percentil_90:,.0f} USD")
print(f"*** Profesionales en Top 10%: {len(top_10_percent)}\n")

print("[Distribución Top 10%]\n")
print("Ubicación:")
print(top_10_percent['es_caba'].value_counts(normalize=True).mul(100).round(1).sort_values(ascending=False))
print("\nTipo de Contrato:")
print(top_10_percent['tipo_contrato'].value_counts(normalize=True).mul(100).round(1).sort_values(ascending=False))
print("\nSeniority:")
print(top_10_percent['seniority'].value_counts(normalize=True).mul(100).round(1).sort_values(ascending=False))
print("\nTop 5 Puestos:")
print(top_10_percent['puesto'].value_counts().head(5))
print(f"\nExperiencia promedio: {top_10_percent['experiencia'].mean():.1f} años")
print(f"Edad promedio: {top_10_percent['edad'].mean():.1f} años")
print(f"Antigüedad promedio: {top_10_percent['antiguedad'].mean():.1f} años\n")

# ====================================================================
# Trayectoria profesional y salto salarial
# ====================================================================
print("Analizando la evolución salarial por experiencia para detectar posibles 'saltos' de carrera.\n")

# El "salto salarial" en la trayectoria profesional

De a poco vemos emerger la infomación relevante. Una vez que analizamos las variables anteriores, podemos abordar la cuestión de la "trayectoria profesional" para encontrar el momento del **salto salarial**, _¿suele tardar en llegar?_, _¿llega en algún momento o tiende más a bien a ser estable?_. Esto es lo que intentamos responder a continuación.

In [ ]:
# Crear rangos de experiencia
df_analisis['rango_exp'] = pd.cut(
    df_analisis['experiencia'], 
    bins=[0, 2, 5, 8, 12, 100],
    labels=['0-2 años (Junior)', '3-5 años (Semi-Senior)', 
            '6-8 años (Senior)', '9-12 años (Senior+)', '12+ años (Expert)']
)

# Estadísticos por rango de experiencia
trayectoria = df_analisis.groupby('rango_exp', observed=True)['salario_usd'].agg([
    ('Cantidad', 'count'), ('Mínimo', 'min'), ('Q25', lambda x: x.quantile(0.25)),
    ('Mediana', 'median'), ('Media', 'mean'), ('Q75', lambda x: x.quantile(0.75)),
    ('Máximo', 'max'), ('Desv_Est', 'std')
]).round(2)

# Calcular crecimiento salarial entre niveles
trayectoria['Crecimiento_%'] = trayectoria['Mediana'].pct_change() * 100
trayectoria['Acumulado_%'] = ((trayectoria['Mediana'] / trayectoria['Mediana'].iloc[0]) - 1) * 100

print("EVOLUCIÓN SALARIAL POR EXPERIENCIA:\n")
print(trayectoria)

# Mayor salto salarial
crecimiento_limpio = trayectoria['Crecimiento_%'].dropna()
if len(crecimiento_limpio) > 0:
    mayor_salto = crecimiento_limpio.idxmax()
    valor_salto = crecimiento_limpio.max()
    print(f"\nMAYOR SALTO SALARIAL: {mayor_salto} (+{valor_salto:.1f}%)\n")

# Evolución del salario a través de los años
Podemos visualizar la evolución del salario mediano a lo largo de los años de experiencia para observar tendencias y patrones en el crecimiento salarial. También podemos analizar la distribución de salarios en diferentes rangos de experiencia para ver cómo varía el salario entre profesionales con diferentes niveles.

In [ ]:
# ====================================================================
# 3. Visualizaciones de trayectoria
# ====================================================================
print("[GENERANDO VISUALIZACIONES DE TRAYECTORIA...]\n")

# Evolución mediana con rango Q25-Q75
fig = go.Figure()

# Área de dispersión (Q25-Q75)
fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))
fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(99, 110, 250, 0.2)',
    fill='tonexty',
    name='Rango Q25-Q75',
    hovertemplate='<b>%{x}</b><br>Q25-Q75: $%{y:,.0f}<extra></extra>'
))

# Línea de mediana
fig.add_trace(go.Scatter(
    x=trayectoria.index.astype(str),
    y=trayectoria['Mediana'],
    mode='lines+markers+text',
    name='Salario Mediano',
    line=dict(color='#636EFA', width=3),
    marker=dict(size=12, color='#636EFA'),
    text=trayectoria['Mediana'].round(0),
    texttemplate='$%{text:,.0f}',
    textposition='top center',
    hovertemplate='<b>%{x}</b><br>Mediana: $%{y:,.0f}<extra></extra>'
))

fig.update_layout(
    title='Trayectoria Salarial: Evolución por Años de Experiencia',
    xaxis_title='Rango de Experiencia',
    yaxis_title='Salario (USD)',
    height=600,
    hovermode='x unified',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

# Gráfico de crecimiento porcentual
fig_growth = go.Figure()
fig_growth.add_trace(go.Bar(
    x=trayectoria.index.astype(str),
    y=trayectoria['Crecimiento_%'],
    name='Crecimiento vs Nivel Anterior',
    marker_color='#636EFA',
    text=trayectoria['Crecimiento_%'].round(1),
    texttemplate='%{text:.1f}%',
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Crecimiento: %{y:.1f}%<extra></extra>'
))
fig_growth.update_layout(
    title='Crecimiento Salarial entre Niveles de Experiencia',
    xaxis_title='Rango de Experiencia',
    yaxis_title='Crecimiento (%)',
    height=500,
    showlegend=False
)
fig_growth.show()

# Visualizaciones comparativas
Ahora bien, una vez que tenemos toda esta información, podemos intentar combinar las variables para encontrar la combinación ganadora que maximiza el salario.

In [ ]:
# ====================================================================
# 4. VISUALIZACIONES COMPARATIVAS
# ====================================================================
print("="*70)
print("VISUALIZACIONES COMPARATIVAS")
print("="*70 + "\n")

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Crear subplots 3x3
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Ubicación', 'Tipo de Contrato', 'Dedicación',
                    'Modalidad', 'Seniority', 'Forma de Pago',
                    'Tamaño Empresa', 'Tiene Equipo', 'Top 5 Puestos'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Función genérica para agregar barras
def agregar_bar(fig, df_stats, col_y, row, col, color):
    fig.add_trace(go.Bar(
        x=df_stats.index,
        y=df_stats[col_y],
        text=df_stats[col_y].round(0),
        texttemplate='$%{text:,.0f}',
        textposition='outside',
        marker_color=color,
        showlegend=False,
        hovertemplate='<b>%{x}</b><br>Salario: $%{y:,.0f}<extra></extra>'
    ), row=row, col=col)
    return fig

# Colores consistentes
colores = ['#FF6B6B', '#4ECDC4', '#95E1D3', '#F38181', 
           '#AA96DA', '#FCBAD3', '#FFD93D', '#A8D8EA', '#FFAAA5']

# Agregar barras
fig = agregar_bar(fig, ubicacion_stats, 'Mediana', 1, 1, colores[0])
fig = agregar_bar(fig, contrato_stats, 'Mediana', 1, 2, colores[1])
fig = agregar_bar(fig, dedicacion_stats, 'Mediana', 1, 3, colores[2])
fig = agregar_bar(fig, modalidad_stats, 'Mediana', 2, 1, colores[3])
fig = agregar_bar(fig, seniority_stats, 'Mediana', 2, 2, colores[4])
fig = agregar_bar(fig, pago_stats, 'Mediana', 2, 3, colores[5])
fig = agregar_bar(fig, empresa_stats, 'Mediana', 3, 1, colores[6])
fig = agregar_bar(fig, equipo_stats, 'Mediana', 3, 2, colores[7])

# Top 5 Puestos
top_5_puestos = puesto_stats.head(5)
fig = agregar_bar(fig, top_5_puestos, 'Mediana', 3, 3, colores[8])

# Ajustes globales de layout
fig.update_yaxes(title_text="Salario (USD)")
fig.update_xaxes(tickangle=-45, tickfont=dict(size=9))

fig.update_layout(
    height=1400,
    title_text="Salarios Medianos por Variable Clave",
    title_font_size=16,
    showlegend=False,
    margin=dict(t=100, b=50, l=50, r=50)
)

fig.show()

# ====================================================================
# TOP 10 COMBINACIONES (Ubicación + Contrato + Dedicación)
# ====================================================================
print("\n[TOP 10 COMBINACIONES]\n")

combinaciones = df_analisis.groupby(['es_caba', 'tipo_contrato', 'dedicacion'])['salario_usd'].agg([
    ('Media', 'mean'),
    ('Mediana', 'median'),
    ('Cantidad', 'count')
]).round(2)

# Filtrar combinaciones con al menos 3 registros y ordenar por Mediana
combinaciones_filtradas = combinaciones[combinaciones['Cantidad'] >= 3] \
    .sort_values(['Mediana', 'Cantidad'], ascending=[False, False]) \
    .head(10)

print("Top 10 Combinaciones (Ubicación + Contrato + Dedicación):")
print(combinaciones_filtradas)
print()



# La combinación ganadora
Después de todo este análisis, podemos finalmente presentar la combinación ganadora que maximiza el salario. Esta combinación se basa en las variables que hemos analizado y las conclusiones que hemos extraído a lo largo de este "análisis exploratorio de datos" (EDA).

In [ ]:
# 5. RESUMEN EJECUTIVO
print("RESUMEN EJECUTIVO: LA COMBINACIÓN GANADORA")
print("="*70 + "\n")

# Perfil del trabajador ideal
resumen = f"""
PERFIL DEL TRABAJADOR IDEAL (MÁXIMOS SALARIOS)

[1] UBICACIÓN: {mejor_ubicacion}
    Mediana: ${ubicacion_stats.loc[mejor_ubicacion, 'Mediana']:,.0f} USD

[2] TIPO DE CONTRATO: {mejor_contrato}
    Mediana: ${contrato_stats.loc[mejor_contrato, 'Mediana']:,.0f} USD

[3] DEDICACIÓN: {mejor_dedicacion}
    Mediana: ${dedicacion_stats.loc[mejor_dedicacion, 'Mediana']:,.0f} USD

[4] MODALIDAD: {mejor_modalidad}
    Mediana: ${modalidad_stats.loc[mejor_modalidad, 'Mediana']:,.0f} USD

[5] SENIORITY: {mejor_seniority}
    Mediana: ${seniority_stats.loc[mejor_seniority, 'Mediana']:,.0f} USD

[6] PUESTO: {mejor_puesto}
    Mediana: ${puesto_stats.loc[mejor_puesto, 'Mediana']:,.0f} USD

[7] FORMA DE PAGO: {mejor_pago}
    Mediana: ${pago_stats.loc[mejor_pago, 'Mediana']:,.0f} USD

[8] TAMAÑO EMPRESA: {mejor_empresa}
    Mediana: ${empresa_stats.loc[mejor_empresa, 'Mediana']:,.0f} USD

[9] LIDERAZGO: {mejor_equipo}
    Mediana: ${equipo_stats.loc[mejor_equipo, 'Mediana']:,.0f} USD

{'-'*50}
EVOLUCIÓN ESPERADA DEL SALARIO
{'-'*50}
"""

# Construir evolución salarial dinámicamente
for idx, fila in trayectoria.iterrows():
    crecimiento = fila['Crecimiento_%']
    resumen += f"{idx}: ${fila['Mediana']:,.0f} USD"
    if not pd.isna(crecimiento):
        resumen += f" (+{crecimiento:.1f}%)"
    resumen += "\n"

resumen += f"""
{'-'*50}
RECOMENDACIONES PARA MAXIMIZAR SALARIO
{'-'*50}
1. Ubicarse en {mejor_ubicacion}
2. Buscar contratos tipo {mejor_contrato}
3. Trabajar {mejor_dedicacion}
4. Priorizar modalidad {mejor_modalidad}
5. Desarrollarse hasta {mejor_seniority}
6. Especializarse en roles como {mejor_puesto}
7. Negociar forma de pago {mejor_pago}
8. Apuntar a empresas {mejor_empresa}
9. Desarrollar capacidad de liderazgo ({mejor_equipo})
10. Acumular ~{top_10_percent['experiencia'].mean():.0f} años de experiencia

Objetivo: Alcanzar Top 10% (${percentil_90:,.0f}+ USD/mes)
"""

print(resumen)

# Machine Learning: Random Forest Regressor
En esta parte del trabajo, buscamos ir más allá del análisis descriptivo y explorar la importancia relativa de las variables en la predicción del salario utilizando técnicas de Machine Learning. Nuestro objetivo es identificar cuáles son las características más influyentes que determinan el salario de los profesionales en el sector IT y ver si existe una coincidencia entre estas variables y las que hemos identificado en nuestro análisis previo como parte de la "combinación ganadora". Creemos que al sumar este enfoque, podemos obtener una visión más completa y robusta de los factores que impactan en el salario, validando o complementando nuestras conclusiones anteriores con un análisis basado en datos y modelos predictivos.

## Importancia de las variables
En primer lugar, debemos preparar los datos para el modelo de Machine Learning. Esto incluye seleccionar las variables relevantes, manejar valores faltantes y dividir los datos en conjuntos de entrenamiento y prueba. El modelo que vamos a aplicar es un Random Forest Regressor, muy utilizado para problemas de regresión que puede manejar tanto variables numéricas como categóricas.

In [ ]:
# 6. MACHINE LEARNING: IMPORTANCIA DE VARIABLES

def categorizar_puesto(puesto):
    if pd.isna(puesto):
        return 'Other'
    puesto = unidecode(str(puesto).lower().strip())
    if re.search(r'developer|desarrollador|programador|frontend|backend|fullstack|mobile|web|software engineer', puesto):
        return 'Developer'
    elif re.search(r'data|bi |business intelligence|analytics|analista de datos|machine learning|ciencia de datos|scientist', puesto):
        return 'Data & Analytics'
    elif re.search(r'manager|management|lider|leader|director|gerente|vp|c-level|head|ceo|cto|cio', puesto):
        return 'Management'
    elif re.search(r'devops|sysadmin|infraestructura|infrastructure|sre|networking|cloud|seguridad|security|systems', puesto):
        return 'Infra/Ops/Security'
    elif re.search(r'qa|testing|tester|automation|quality', puesto):
        return 'QA / Testing'
    elif re.search(r'product|pm|po|product owner|product manager', puesto):
        return 'Product'
    elif re.search(r'scrum|agile|coach', puesto):
        return 'Agile / Scrum'
    elif re.search(r'ux|ui|designer|disenador|design', puesto):
        return 'UX/UI Design'
    elif re.search(r'architect|arquitecto', puesto):
        return 'Architect'
    elif re.search(r'consultor|consultant|functional|specialist', puesto):
        return 'Consultant'
    else:
        return 'Other'

df_clean['puesto_categoria'] = df_clean['puesto'].apply(categorizar_puesto)
print("[INFO] Categorias de puestos creadas")
print(df_clean['puesto_categoria'].value_counts(), "\n")

# ------------------------------------------------------------
# 2. Simplificar tamaño de empresa
# ------------------------------------------------------------
def simplificar_tamano_empresa(tamano):
    if pd.isna(tamano):
        return 'Desconocido'
    tamano = str(tamano)
    if any(x in tamano for x in ['De 2 a', 'De 11', 'De 51', '1 (solamente']):
        return 'Pequeña (1-100)'
    elif any(x in tamano for x in ['De 101', 'De 201', 'De 501']):
        return 'Mediana (101-1000)'
    else:
        return 'Grande (1000+)'

df_clean['tamano_empresa'] = df_clean['tamano_empresa'].apply(simplificar_tamano_empresa)
print(f"[INFO] Tamaño empresa simplificado:")
print(df_clean['tamano_empresa'].value_counts(), "\n")

# ------------------------------------------------------------
# 3. Preparar datos para modelo
# ------------------------------------------------------------
features_list = ['experiencia', 'antiguedad', 'edad', 'seniority', 'modalidad',
                 'es_caba', 'tipo_contrato', 'tamano_empresa', 'puesto_categoria',
                 'forma_pago', 'tiene_equipo']

features_list = [col for col in features_list if col in df_clean.columns]
df_model = df_clean[features_list + ['salario_usd']].dropna()
print(f"[INFO] Registros iniciales: {len(df_model)}")

# ------------------------------------------------------------
# 4. Limpieza de datos
# ------------------------------------------------------------
# Eliminar infinitos y NaN remanentes
df_model = df_model.replace([np.inf, -np.inf], np.nan).dropna()
print(f"Después de eliminar infinitos: {len(df_model)} registros")

# Eliminar salarios <= 0
df_model = df_model[df_model['salario_usd'] > 0]
print(f"Después de eliminar salarios <= 0: {len(df_model)} registros")

# Filtrar outliers usando percentiles 0.5 y 99.5
Q1 = df_model['salario_usd'].quantile(0.005)
Q99 = df_model['salario_usd'].quantile(0.995)
print(f"Rango válido: ${Q1:,.0f} - ${Q99:,.0f} USD")

df_model = df_model[(df_model['salario_usd'] >= Q1) & (df_model['salario_usd'] <= Q99)]
print(f"Después de filtrar outliers extremos: {len(df_model)} registros")

# Validar rangos de variables
df_model = df_model[
    (df_model['experiencia'] >= 0) & (df_model['experiencia'] <= 50) &
    (df_model['edad'] >= 18) & (df_model['edad'] <= 80) &
    (df_model['antiguedad'] >= 0) & (df_model['antiguedad'] <= 50)
]
print(f"Después de validar rangos de variables: {len(df_model)} registros\n")

print(f"[INFO] Registros finales para modelo: {len(df_model)}")
print(f"[INFO] Variables predictoras: {len(features_list)}")
print(f"[INFO] Salario USD - Min: ${df_model['salario_usd'].min():,.0f} | Max: ${df_model['salario_usd'].max():,.0f}")
print(f"[INFO] Salario USD - Media: ${df_model['salario_usd'].mean():,.0f} | Mediana: ${df_model['salario_usd'].median():,.0f}\n")

# ------------------------------------------------------------
# 5. Encoding y verificación
# ------------------------------------------------------------
X = pd.get_dummies(df_model[features_list], drop_first=True)
y = df_model['salario_usd']
feature_names = X.columns.tolist()

X = X.apply(pd.to_numeric, errors='coerce')

# Validación de NaN e infinitos
X = X.fillna(0)
nan_count_x = X.isna().sum().sum()
inf_count_x = np.isinf(X.select_dtypes(include=[np.number]).values).sum()
print(f"[VALIDACIÓN] NaN en X: {nan_count_x} | Inf: {inf_count_x}")
print(f"[VALIDACIÓN] NaN en y: {y.isna().sum()} | Inf: {np.isinf(y.values).sum()}\n")

# ------------------------------------------------------------
# 6. Train/Test split
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"[MODELO] Entrenamiento: {len(X_train)} | Test: {len(X_test)}")

# ------------------------------------------------------------
# 7. Entrenamiento Random Forest
# ------------------------------------------------------------
model = RandomForestRegressor(
    n_estimators=150,         
    max_depth=None,             
    min_samples_split=20,      
    min_samples_leaf=10,       
    max_features='sqrt',       
    max_samples=0.8,           
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("[MODELO] Entrenado exitosamente\n")

# ------------------------------------------------------------
# 8. Métricas del modelo
# ------------------------------------------------------------
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("MÉTRICAS DEL MODELO")
print(f"MAE Train: ${mae_train:,.0f} | MAE Test: ${mae_test:,.0f}")
print(f"RMSE Test: ${rmse_test:,.0f}")
print(f"R² Train: {r2_train:.4f} | R² Test: {r2_test:.4f}")

diff_r2 = r2_train - r2_test
if diff_r2 > 0.1:
    print(f"\nAdvertencia: Posible overfitting (diferencia R²: {diff_r2:.3f})")
else:
    print(f"\nEl modelo generaliza bien (diferencia R²: {diff_r2:.3f})\n")

# ------------------------------------------------------------
# 9. Validación cruzada
# ------------------------------------------------------------
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
cv_mae = -cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
print("VALIDACIÓN CRUZADA (5-FOLD)")
print(f"R² promedio: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
print(f"R² por fold: {[f'{score:.3f}' for score in cv_scores]}")
print(f"MAE promedio: ${cv_mae.mean():,.0f} (±${cv_mae.std():,.0f})\n")

# ------------------------------------------------------------
# 10. Importancia de variables
# ------------------------------------------------------------
importancia = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': model.feature_importances_
}).sort_values('Importancia', ascending=False)

fig = px.bar(importancia, x='Variable', y='Importancia', title='Importancia de Variables', text='Importancia')
fig.update_layout(height=500, xaxis_tickangle=-45, showlegend=False)
fig.show()

## Visualización: predicciones vs. realidad
El modelo entrenado nos permite hacer predicciones sobre los salarios basándonos en las características de los profesionales. Para evaluar el rendimiento del modelo, podemos comparar las predicciones con los valores reales de salario en el conjunto de prueba. Una forma efectiva de visualizar esta comparación es mediante un gráfico de dispersión (scatter plot), donde cada punto representa un profesional, con su salario real en el eje Y y su salario predicho por el modelo en el eje X.

In [ ]:
# 6.1 VISUALIZACIÓN: PREDICCIONES VS REALES

n_points = len(y_test)
if n_points > 500:
    sample_indices = np.random.choice(n_points, 500, replace=False)
    y_test_sample = y_test.iloc[sample_indices]
    y_pred_sample = y_pred_test[sample_indices]
else:
    y_test_sample = y_test
    y_pred_sample = y_pred_test

fig = go.Figure()

# Scatter plot de predicciones vs reales
fig.add_trace(go.Scatter(
    x=y_test_sample,
    y=y_pred_sample,
    mode='markers',
    marker=dict(
        color=y_test_sample,
        colorscale='Viridis',
        size=6,
        opacity=0.6,
        colorbar=dict(title="Salario Real (USD)", len=0.7),
        line=dict(width=0.5, color='white')
    ),
    name='Predicciones',
    hovertemplate='<b>Real:</b> $%{x:,.0f}<br><b>Predicho:</b> $%{y:,.0f}<extra></extra>'
))

# Línea de predicción perfecta
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='Predicción Perfecta',
    hoverinfo='skip'
))

# Bandas de error ±MAE
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val + mae_test, max_val + mae_test],
    mode='lines',
    line=dict(color='orange', dash='dot', width=1),
    name=f'Error +${mae_test:,.0f}',
    hoverinfo='skip'
))
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val - mae_test, max_val - mae_test],
    mode='lines',
    line=dict(color='orange', dash='dot', width=1),
    name=f'Error -${mae_test:,.0f}',
    hoverinfo='skip'
))

fig.update_layout(
    title=dict(
        text=f'Predicciones vs Valores Reales<br><sub>R² = {r2_test:.3f} | MAE = ${mae_test:,.0f} USD</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(title='Salario Real (USD)', tickformat='$,.0f', gridcolor='lightgray'),
    yaxis=dict(title='Salario Predicho (USD)', tickformat='$,.0f', gridcolor='lightgray'),
    height=700,
    hovermode='closest',
    plot_bgcolor='white',
    legend=dict(orientation="v", yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.8)"),
    margin=dict(t=100, b=50, l=80, r=50)
)

if n_points > 500:
    fig.add_annotation(
        text=f"Nota: Mostrando 500 de {n_points} puntos para mejor visualización",
        xref="paper", yref="paper",
        x=0.5, y=-0.12,
        showarrow=False,
        font=dict(size=10, color="gray")
    )

fig.show()

## Cálculo de la importancia de las variables
En este caso, el modelo nos proporciona una medida de la importancia de cada variable en la predicción del salario. Podemos ordenar estas importancias y visualizarlas para identificar cuáles son las características más influyentes.

In [ ]:
# 6.2 IMPORTANCIA DE VARIABLES

importances = model.feature_importances_
feature_imp_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia_%': importances * 100
}).sort_values(by='Importancia_%', ascending=False)

print("TOP 15 VARIABLES MÁS IMPORTANTES:\n")
print(feature_imp_df.head(15), "\n")

# Agrupar variables por categoría
def extraer_categoria(var):
    if 'seniority_' in var: return 'Seniority'
    elif 'puesto_categoria_' in var: return 'Puesto/Rol'
    elif 'modalidad_' in var: return 'Modalidad'
    elif 'tipo_contrato_' in var: return 'Tipo Contrato'
    elif 'tamano_empresa_' in var: return 'Tamano Empresa'
    elif 'forma_pago_' in var: return 'Forma de Pago'
    elif 'es_caba_' in var: return 'Ubicacion'
    elif 'tiene_equipo_' in var: return 'Liderazgo'
    elif var in ['experiencia', 'antiguedad', 'edad']: return var.capitalize()
    else: return 'Otros'

feature_imp_df['Categoria'] = feature_imp_df['Variable'].apply(extraer_categoria)
imp_agrupada = feature_imp_df.groupby('Categoria')['Importancia_%'].sum().sort_values(ascending=False)

print("IMPORTANCIA AGRUPADA POR CATEGORÍA:\n")
print(imp_agrupada.to_frame(), "\n")

# Visualización Top 20 Variables
fig = px.bar(
    feature_imp_df.head(20).sort_values(by='Importancia_%', ascending=True),
    x='Importancia_%',
    y='Variable',
    text='Importancia_%',
    color='Importancia_%',
    color_continuous_scale='Viridis',
    title='Top 20 Variables Más Importantes para Predecir el Salario',
    labels={'Importancia_%': 'Importancia (%)', 'Variable': 'Variable'}
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=800, showlegend=False)
fig.show()

# Visualización: Importancia Agrupada por Categoría
fig = px.bar(
    x=imp_agrupada.values,
    y=imp_agrupada.index,
    orientation='h',
    text=imp_agrupada.values,
    color=imp_agrupada.values,
    color_continuous_scale='Plasma',
    title='Importancia Agrupada por Categoría de Variable',
    labels={'x': 'Importancia Total (%)', 'y': 'Categoría'}
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=600, showlegend=False)
fig.show()

## Análisis de residuos
Finalmente, podemos analizar los residuos del modelo, es decir, la diferencia entre los salarios reales y los predichos. Esto nos permite identificar patrones en los errores del modelo y entender mejor su rendimiento.

In [ ]:
# 6.3 ANÁLISIS DE RESIDUOS

residuos = y_test - y_pred_test

print("ESTADÍSTICAS DE LOS RESIDUOS:")
print(f"Media: ${residuos.mean():,.0f} USD")
print(f"Mediana: ${residuos.median():,.0f} USD")
print(f"Desviación Estándar: ${residuos.std():,.0f} USD\n")

# Histograma de residuos
fig = px.histogram(
    residuos,
    nbins=50,
    title='Distribución de Errores del Modelo',
    labels={'value': 'Error (USD)', 'count': 'Frecuencia'},
    color_discrete_sequence=['#636EFA']
)
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Error = 0", annotation_position="top right")
fig.update_layout(height=500, showlegend=False)
fig.show()

# Conclusión de nuestro análisis
A modo de cierre, luego del EDA ("Análisis exploratorio de Datos") y del análisis utilizando Machine Learning, podemos llegar a la conclusión de que la mejor combinación de variables para maximizar el salario en el sector IT es la siguiente: en primer lugar, la experiencia laboral. Podríamos decir que "cuanto antes, mejor", es decir, comenzar a trabajar lo antes posible para acumular experiencia, incluso mientras se está estudiando, ya que no parece ser un mercado laboral que exija títulos universitarios para acceder a mejores salarios. La mejora constante sí es algo necesario pero no necesariamente a través de títulos formales. La dolarización del salario también es un factor clave, ya que los salarios en dólares tienden a ser más altos en comparación con los salarios en moneda local. Por último, algo que también está vinculado con la seniority/experiencia es el liderazgo, aquellas personas con "gente a cargo" que dirigen equipos suelen tener salarios más altos. Esta última variable se vincula directamente con las llamadas "soft skills", lo que coincide con la tendencia actual del mercado laboral a valorar cada vez más estas habilidades interpersonales y de gestión.

In [ ]:
# 7. CONCLUSIONES FINALES

top_cat = imp_agrupada.index[0]
top_val = imp_agrupada.iloc[0]

# Asegurarse de que existan al menos 3 variables para mostrar
top_variables = feature_imp_df.head(3)

print("CONCLUSIONES DEL MODELO PREDICTIVO\n")
print(f"""
El modelo Random Forest revela que:

* Precisión (R² Test): {r2_test:.3f} ({r2_test*100:.1f}% de la variabilidad salarial explicada)
* Precisión (R² CV): {cv_scores.mean():.3f} (validación cruzada confirma estabilidad)
* Error promedio (MAE): ${mae_test:,.0f} USD/mes ±${cv_mae.std():,.0f}
* Overfitting: {'Controlado' if diff_r2 < 0.2 else 'Presente'} (diferencia R²: {diff_r2:.3f})
* Factor más determinante: {top_cat} ({top_val:.1f}% de importancia)

Top {len(top_variables)} variables individuales más importantes:
""")


for i, row in top_variables.iterrows():
    print(f"{i+1}. {row['Variable']}: {row['Importancia_%']:.2f}%")

print(f"""
Implicación práctica:
Para maximizar el salario, el trabajador debe enfocarse principalmente en optimizar su 
{top_cat.lower()}, seguido de las variables individuales identificadas arriba.
""")